In [ ]:
import pandas as pd
# Import Excel with columns (Variables Explicatives) expected for this script
df = pd.read_excel('extract-dpe-cut.xlsx')

In [ ]:
ls_variables_explicatives = ['Année_construction','Surface_habitable_logement','Coût_total_5_usages','Coût_ECS','Coût_chauffage','Coût_éclairage','Coût_auxiliaires','Coût_refroidissement']
X = df[ls_variables_explicatives]
y = df['Etiquette_DPE']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

# Exemple de données X et y (assurez-vous que X et y sont définis dans votre code)
# X = pd.DataFrame(...)  # Remplacez avec vos données
# y = ...

# Encodage des étiquettes
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Division en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Identification des colonnes numériques et catégorielles
# numeric_features = ['Surface_habitable_logement', 'Coût_total_5_usages',
#                     'Coût_ECS', 'Coût_chauffage', 'Coût_éclairage', 'Coût_auxiliaires', 'Coût_refroidissement','Année_construction']
numeric_features = ['Année_construction','Surface_habitable_logement','Coût_total_5_usages','Coût_ECS','Coût_chauffage','Coût_éclairage','Coût_auxiliaires','Coût_refroidissement']


# Préprocesseur pour imputer, normaliser les données numériques et encoder les catégories
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),  # Impute avec la moyenne pour les numériques
            ('scaler', StandardScaler())  # Normalisation des données numériques
        ]), numeric_features),
    ])

# Pipeline avec prétraitement et modèle de classification
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=5, max_depth=30))
])

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Évaluation du modèle
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [ ]:
import joblib
joblib.dump((pipeline, label_encoder),'modele_classif2.pkl', compress=3)